<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/modelos_tradicionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/2010.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

# Agora, 'dados' contém os dados do arquivo XLSX


In [41]:
# Exibe as primeiras linhas do DataFrame
print(dados.head())


   ID Código    Setor EconômicoB3     VM30Abr11   PLDez2010  LLDez2010  \
0   5  EALT4     Bens industriais  6.072750e+04     45809.0     5927.0   
1  21  ALPA4      Consumo cíclico  3.893923e+06   1310475.0   306341.0   
2  26  ABEV3  Consumo não cíclico  1.422082e+08  24361863.0  7561383.0   
3  28  AMER3      Consumo cíclico  2.382110e+06    225945.0    33587.0   
4  29  CBEE3    Utilidade pública  4.773374e+06   1583469.0   216092.0   

    ATDez2010   PTDez2010  VM30Abr11esc  PLDez2010esc  LLDez2010esc  \
0    237963.0    192154.0      0.424087      0.319904      0.041391   
1   2246603.0    936128.0      2.226423      0.749289      0.175156   
2  42678300.0  18316437.0      3.546250      0.607512      0.188558   
3   3212014.0   2986069.0      1.024863      0.097209      0.014450   
4   4313606.0   2730137.0      1.040166      0.345054      0.047089   

   ATDez2010esc  PTDez2010esc   ATDez2009  
0      1.661799      1.341895    143196.0  
1      1.284537      0.535249   1748959.

REGRESSÃO SEM ESCALA

tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [3]:
from scipy.stats.mstats import winsorize

nivel_winsorizacao = 0.01  # 1% de winsorização

# Aplica a winsorização às variáveis
dados['PLDez2010_winsorizada'] = winsorize(dados['PLDez2010esc'], limits=[nivel_winsorizacao, nivel_winsorizacao])
dados['LLDez2010_winsorizada'] = winsorize(dados['LLDez2010esc'], limits=[nivel_winsorizacao, nivel_winsorizacao])
dados['VM30Abr11_winsorizada'] = winsorize(dados['VM30Abr11esc'], limits=[nivel_winsorizacao, nivel_winsorizacao])
# Variáveis winsorizadas agora contêm os valores winsorizados

In [4]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PLDez2010_winsorizada', 'LLDez2010_winsorizada']]
y = dados['VM30Abr11_winsorizada']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Visualize os resultados do modelo
print(modelo.summary())

                              OLS Regression Results                             
Dep. Variable:     VM30Abr11_winsorizada   R-squared:                       0.102
Model:                               OLS   Adj. R-squared:                  0.093
Method:                    Least Squares   F-statistic:                     10.54
Date:                   Thu, 14 Sep 2023   Prob (F-statistic):           4.63e-05
Time:                           15:49:51   Log-Likelihood:                -450.56
No. Observations:                    188   AIC:                             907.1
Df Residuals:                        185   BIC:                             916.8
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------


Em relação aos resultados apresentados no seu modelo de regressão:

O valor do teste de Durbin-Watson é 1.676, o que sugere uma leve autocorrelação positiva nos resíduos, mas não muito forte.
O valor-p associado ao teste de Jarque-Bera é praticamente zero (0.00), o que indica que os resíduos não seguem uma distribuição normal.

Interpretação DW:

Valor DW ≈ 2: Ausência de autocorrelação significativa nos resíduos (boa notícia).
Valor DW < 2: Autocorrelação positiva nos resíduos (erros adjacentes são correlacionados positivamente).
Valor DW > 2: Autocorrelação negativa nos resíduos (erros adjacentes são correlacionados negativamente).

teste de heterocedasticidade de White. Se o valor-p for menor que um nível de significância escolhido (por exemplo, 0,05), você pode rejeitar a hipótese nula de homocedasticidade

In [44]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

# Calcule os resíduos do modelo
residuos = modelo.resid

# Realize o teste de White para heterocedasticidade
teste_white = het_white(residuos, X)

# Imprima os resultados do teste
print("Estatística do teste de White:", teste_white[0])
print("Valor-p do teste de White:", teste_white[1])

Estatística do teste de White: 113.73736510497207
Valor-p do teste de White: 6.642361768007011e-23


erro absoluto médio (MAE) e erro médio quadrático (MSE)

In [45]:
import statsmodels.api as sm
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Faça previsões com o modelo
previsoes = modelo.predict(X)

# Calcule o Median Absolute Error (MedAE)
medae = np.median(np.abs(y - previsoes))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

# Calcule o Erro Absoluto Médio (MAE)
mae = np.mean(np.abs(y - previsoes))

# Calcule o Erro Médio Quadrático (MSE)
mse = np.mean((y - previsoes) ** 2)

# Calcule a Raiz do Erro Quadrático Médio (RMSE)
rmse = np.sqrt(mse)

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)
print("Erro Absoluto Médio (MAE):", mae)
print("Erro Médio Quadrático (MSE):", mse)
print("Raiz do Erro Quadrático Médio (RMSE):", rmse)


Median Absolute Error (MedAE): 0.907334942505559
MedAE escalado pelo valor (ou preço) em percentagem: 56.79161460647346
Erro Absoluto Médio (MAE): 1.3779344200822823
Erro Médio Quadrático (MSE): 7.066169255778478
Raiz do Erro Quadrático Médio (RMSE): 2.6582267126372945


Nesse caso específico, o valor de 17.74% indica que, em média, as previsões do modelo estão erradas em cerca de 17.74% do valor médio (ou preço) da variável de resposta. Isso ajuda a ter uma ideia do tamanho relativo dos erros do modelo em relação ao valor médio dos dados.

minimos quadrados ponderados, com pesos com base na variância dos erros

In [46]:
import statsmodels.api as sm
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Calcule os resíduos (erros)
residuos = modelo.resid

# Calcule a variância dos resíduos
variancia_residuos = np.var(residuos)

# Calcule os pesos com base na variância dos resíduos (inverso da variância)
pesos = 1 / variancia_residuos

# Crie o modelo de regressão linear ponderada (WLS) com os pesos calculados
modelo_wls = sm.WLS(y, X, weights=pesos).fit()

# Visualize os resultados do modelo ponderado
print(modelo_wls.summary())


                            WLS Regression Results                            
Dep. Variable:              VM30Abr11   R-squared:                       0.963
Model:                            WLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     2411.
Date:                Thu, 14 Sep 2023   Prob (F-statistic):          3.13e-133
Time:                        13:30:10   Log-Likelihood:                -3216.0
No. Observations:                 188   AIC:                             6438.
Df Residuals:                     185   BIC:                             6448.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.752e+06   4.87e+05      3.595      0.0

In [47]:
import statsmodels.api as sm
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear ponderada (WLS) com os pesos calculados
modelo_wls = sm.WLS(y, X, weights=pesos).fit()

# Faça previsões com o modelo
previsoes = modelo_wls.predict(X)

# Calcule o Median Absolute Error (MedAE)
medae = np.median(np.abs(y - previsoes))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

# Calcule o Erro Absoluto Médio (MAE)
mae = np.mean(np.abs(y - previsoes))

# Calcule o Erro Médio Quadrático (MSE)
mse = np.mean((y - previsoes) ** 2)

# Calcule a Raiz do Erro Quadrático Médio (RMSE)
rmse = np.sqrt(mse)

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)
print("Erro Absoluto Médio (MAE):", mae)
print("Erro Médio Quadrático (MSE):", mse)
print("Raiz do Erro Quadrático Médio (RMSE):", rmse)


Median Absolute Error (MedAE): 1710616.3343900286
MedAE escalado pelo valor (ou preço) em percentagem: 20.74108453982559
Erro Absoluto Médio (MAE): 2842520.7540884106
Erro Médio Quadrático (MSE): 42280004974664.125
Raiz do Erro Quadrático Médio (RMSE): 6502307.665334217


ESTIMAÇÃO FORA DA AMOSTRA MQO

In [ ]:
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependentes
X = dados[['PL Dez 2010', 'LLDez 2010']]
y = dados['VM30Abr11']

# Converta os dados para matrizes NumPy
X = X.to_numpy()
y = y.to_numpy()

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X)

# Defina a validação cruzada de 10-fold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lista para armazenar as métricas de MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled_scores = []

# Realize a validação cruzada e calcule as métricas em cada fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    modelo_fold = modelo.fit()
    y_pred = modelo_fold.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scaled_scores.append(medae_scaled)

# Calcule a média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scaled_scores)

# Imprima a média do R² e do MedAE escalado
print(f'Média do R²: {mean_r2_score}')
print(f'Média do MedAE escalado pelo valor (ou preço) em percentagem: {mean_medae_scaled}')


ARVORE DE DECISÃO_dentro da amostra

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Criar o modelo de árvore de decisão com os parâmetros desejados
modelo_arvore = DecisionTreeRegressor(min_samples_split=5, random_state=42)

# Criar o modelo Bagging com 500 árvores
bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=500, bootstrap=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    bagging_model.fit(X_train, y_train)
    y_pred = bagging_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Média do MedAE escalado pelo valor (ou preço) em percentagem: 14.856151860032648
Média do R²: 0.4084875580187829


ARVORE DE DECISÃO FORA DA AMOSTRA

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Dividir os dados em conjunto de treinamento e teste (80% treinamento, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de árvore de decisão com os parâmetros desejados
modelo_arvore = DecisionTreeRegressor(min_samples_split=5, random_state=42)

# Criar o modelo Bagging com 500 árvores
bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=500, bootstrap=True, random_state=42)

# Treinar o modelo no conjunto de treinamento
bagging_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = bagging_model.predict(X_test)

# Calcular o MedAE escalado pelo valor (ou preço) em percentagem
medae = np.median(np.abs(y_test - y_pred))
valor_medio_y = np.mean(y_test)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

# Calcular o R²
r2 = r2_score(y_test, y_pred)

# Exibir resultados
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)
print("R²:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


MedAE escalado pelo valor (ou preço) em percentagem: 18.42446864752979
R²: 0.7630562414726623


SUPORTE VECTOR REGRESSION DENTRO DA AMOSTRA

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Criar o modelo SVR com os parâmetros desejados
svr_model = SVR(kernel='linear', C=1.0)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    svr_model.fit(X_train, y_train)
    y_pred = svr_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


SUPORTE VECTOR REGRESSION FORA DA AMOSTRA

In [9]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PLDez2010', 'LLDez2010']]
y = dados['VM30Abr11']

# Dividir os dados em conjunto de treinamento e teste (80% treinamento, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo SVR com os parâmetros desejados
svr_model = SVR(kernel='linear', C=1.0)

# Treinar o modelo no conjunto de treinamento
svr_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = svr_model.predict(X_test)

# Calcular o MedAE escalado pelo valor (ou preço) em percentagem
medae = np.median(np.abs(y_test - y_pred))
valor_medio_y = np.mean(y_test)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

# Calcular o R²
r2 = r2_score(y_test, y_pred)

# Exibir resultados
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)
print("R²:", r2)


MedAE escalado pelo valor (ou preço) em percentagem: 25.92721366061097
R²: -0.10119344188688628
